# Converting DataFrames to GeoJSON

Data from [City of Berkeley](https://data.cityofberkeley.info/).

The [311 Cases dataset](https://data.cityofberkeley.info/311/311-Cases-COB/bscu-qpbu) represents service request or inquiry calls, emails and online requests submitted to the City of Berkeley.



In [1]:
import pandas as pd, requests, json

In [2]:
# API endpoint for city of Berkeley's 311 calls
# Note endpoint may need updating if it breaks.  See links above to get new endpoint

endpoint_url = 'https://data.cityofberkeley.info/resource/bscu-qpbu.json?$where=within_circle(location, 37.84, -122.27, 1000)'

In [3]:
# fetch the URL and load the data
response = requests.get(endpoint_url)
data = response.json()

In [4]:
# turn the json data into a dataframe and see how many rows and what columns we have
df = pd.DataFrame(data)

df.head()

,case_id,date_opened,case_status,date_closed,request_category,request_subcategory,request_detail,object_type,apn,street_address,city,state,neighborhood,latitude,longitude,location
0,121000169072,2013-11-19T12:03:49.000,Closed,2013-12-19T23:01:03.000,Business License,Rental of Real Property,Application,Property,052 152700300,1835 SIXTY-THIRD ST,Berkeley,CA,Berkeley,37.84832459,-122.26964667,"{'latitude': '37.84832459', 'longitude': '-122..."
1,121000315937,2016-09-26T10:44:25.000,Closed,2016-12-05T07:58:40.000,"Facilities, Electrical & Property Management",Electrical Services,Street Lights - Repair,Property,052 152700700,1819 SIXTY-THIRD ST,Berkeley,CA,Berkeley,37.84825671,-122.27015104,"{'latitude': '37.84825671', 'longitude': '-122..."
2,121000432122,2018-11-20T10:26:45.000,Closed,2018-12-20T13:09:36.000,Refuse and Recycling,Residential,Residential Reminder,Property,052 152700300,1835 SIXTY-THIRD ST,Berkeley,CA,Berkeley,37.84832459,-122.26964667,"{'latitude': '37.84832459', 'longitude': '-122..."
3,121000100320,2012-07-03T11:21:36.000,Closed,2012-07-26T13:12:43.000,Refuse and Recycling,Commercial,Commercial Service Stop,Property,052 152701002,1813 SIXTY-THIRD ST,Berkeley,CA,Berkeley,37.84820439,-122.27055940,"{'latitude': '37.84820439', 'longitude': '-122..."
4,121000207888,2014-08-12T16:58:15.000,Closed,2014-08-14T07:56:02.000,"Streets, Utilities, and Transportation",Clean City Program,Illegal Dumping - City Property,Property,052 152701002,1813 SIXTY-THIRD ST,Berkeley,CA,Berkeley,37.84820439,-122.27055940,"{'latitude': '37.84820439', 'longitude': '-122..."


In [5]:
df.shape

(1000, 16)

In [6]:
# convert lat-long to floats and change address from ALL CAPS to Regular Capitalization
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['street_address'] = df['street_address'].str.title()

In [7]:
# we don't need all those columns - only keep useful ones
useful_cols = ['request_detail', 'request_category', 'latitude', 'longitude', 'street_address', 'case_status']
df_subset = df[useful_cols]

In [8]:
# drop any rows that lack lat/long data
df_geo = df_subset.dropna(subset=['latitude', 'longitude'], axis=0, inplace=False)

print('We have {} geotagged rows'.format(len(df_geo)))
df_geo.tail()

We have 1000 geotagged rows


,request_detail,request_category,latitude,longitude,street_address,case_status
995,Residential Customer Complaint,Refuse and Recycling,37.847933,-122.272577,1719 Sixty-Third St,Closed
996,Illegal Dumping - City Property,"Streets, Utilities, and Transportation",37.848108,-122.274055,1634 Alcatraz Ave,Closed
997,Residential Bulky Pickup,Refuse and Recycling,37.848131,-122.273904,1638 Alcatraz Ave,Open
998,Miscellaneous Internet Request,General Questions/information,37.848131,-122.273904,1638 Alcatraz Ave,Closed
999,Illegal Dumping - City Property,"Streets, Utilities, and Transportation",37.848204,-122.270559,1813 Sixty-Third St,Closed


In [9]:
# what is the distribution of issue types?
df_geo['request_category'].value_counts()

Refuse and Recycling                            557
Streets, Utilities, and Transportation          215
General Questions/information                   163
Parks, Trees and Vegetation                      24
Business License                                  9
Environmental Services and Programs               9
Facilities, Electrical & Property Management      8
Graffiti and Vandalism                            7
Government Activity                               6
Other Account Services and Billing                2
Name: request_category, dtype: int64

Finally, convert each row in the dataframe to a geojson-formatted feature and save the result as a file. The format is pretty simple and you can see it here: http://geojson.org/

In [10]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [11]:
useful_columns = ['street_address', 'request_detail', 'request_category', 'case_status']
geojson_dict = df_to_geojson(df_geo, properties=useful_columns)
geojson_str = json.dumps(geojson_dict, indent=2)

In [12]:
# save the geojson result to a file
output_filename = 'markerpoints.js'
with open(output_filename, 'w') as output_file:
    output_file.write('var dataset = {};'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

1000 geotagged features saved to file
